In [1]:
from __future__ import print_function
import sys
from operator import add
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
.builder\
.appName("Assignment_3")\
.getOrCreate()

In [3]:
input_path = "/Users/kalyani/Documents/Study/Big_Data/a3.csv"

In [4]:
records = spark.sparkContext.textFile(input_path)

In [5]:
records.take(2)

['NAMELAST,NAMEFIRST,NAMEMID,UIN,BDGNBR,ACCESS_TYPE,TOA,POA,TOD,POD,APPT_MADE_DATE,APPT_START_DATE,APPT_END_DATE,APPT_CANCEL_DATE,Total_People,LAST_UPDATEDBY,POST,LASTENTRYDATE,TERMINAL_SUFFIX,visitee_namelast,visitee_namefirst,MEETING_LOC,MEETING_ROOM,CALLER_NAME_LAST,CALLER_NAME_FIRST,CALLER_ROOM,DESCRIPTION,Release_Date',
 'TAJOURIBESSASSI,HANENE,,U22101,,VA,,,,,9/2/2015 0:00,10/1/2015 3:00,10/1/2015 23:59,,1,AR,WIN,9/2/2015 11:38,AR,Pelofsky,Eric,OEOB,226,ROWBERRY,ARIANA,,,1/29/2016']

In [6]:
tokens = records.map(lambda x: x.lower())

In [7]:
type(tokens)

pyspark.rdd.PipelinedRDD

In [8]:
tokens.take(2)

['namelast,namefirst,namemid,uin,bdgnbr,access_type,toa,poa,tod,pod,appt_made_date,appt_start_date,appt_end_date,appt_cancel_date,total_people,last_updatedby,post,lastentrydate,terminal_suffix,visitee_namelast,visitee_namefirst,meeting_loc,meeting_room,caller_name_last,caller_name_first,caller_room,description,release_date',
 'tajouribessassi,hanene,,u22101,,va,,,,,9/2/2015 0:00,10/1/2015 3:00,10/1/2015 23:59,,1,ar,win,9/2/2015 11:38,ar,pelofsky,eric,oeob,226,rowberry,ariana,,,1/29/2016']

In [9]:
tokens = tokens.map(lambda x: x.split(','))

In [10]:
tokens.take(1)

[['namelast',
  'namefirst',
  'namemid',
  'uin',
  'bdgnbr',
  'access_type',
  'toa',
  'poa',
  'tod',
  'pod',
  'appt_made_date',
  'appt_start_date',
  'appt_end_date',
  'appt_cancel_date',
  'total_people',
  'last_updatedby',
  'post',
  'lastentrydate',
  'terminal_suffix',
  'visitee_namelast',
  'visitee_namefirst',
  'meeting_loc',
  'meeting_room',
  'caller_name_last',
  'caller_name_first',
  'caller_room',
  'description',
  'release_date']]

In [11]:
tokens1 = tokens.map(lambda x: (x[0], x[1],x[2], x[19], x[20]))

In [12]:
tokens1.take(2)

[('namelast', 'namefirst', 'namemid', 'visitee_namelast', 'visitee_namefirst'),
 ('tajouribessassi', 'hanene', '', 'pelofsky', 'eric')]

In [13]:
header = tokens1.first()

In [14]:
header

('namelast', 'namefirst', 'namemid', 'visitee_namelast', 'visitee_namefirst')

In [15]:
tokens2 = tokens1.filter(lambda x: x != header)

In [16]:
tokens2.take(5)

[('tajouribessassi', 'hanene', '', 'pelofsky', 'eric'),
 ('bageant', 'laura', 'j', 'baskerville', 'steven'),
 ('broemson', 'earl', 'h', 'baskerville', 'steven'),
 ('jackling jr', 'william', 'c', 'baskerville', 'steven'),
 ('mccrary', 'richard', 'l', 'baskerville', 'steven')]

In [17]:
tokens2.count()

970504

# Filter records where Visitor/Visitee is missing

In [18]:
filtered = tokens2.filter(lambda x: x[3] != "")

In [19]:
type(filtered)

pyspark.rdd.PipelinedRDD

In [20]:
filtered.collect()

[('tajouribessassi', 'hanene', '', 'pelofsky', 'eric'),
 ('bageant', 'laura', 'j', 'baskerville', 'steven'),
 ('broemson', 'earl', 'h', 'baskerville', 'steven'),
 ('jackling jr', 'william', 'c', 'baskerville', 'steven'),
 ('mccrary', 'richard', 'l', 'baskerville', 'steven'),
 ('mulcahy', 'joshua', 'e', 'baskerville', 'steven'),
 ('ryan', 'oliver', 'j', 'baskerville', 'steven'),
 ('smith jr', 'william', 't', 'baskerville', 'steven'),
 ('keeler', 'douglas', 'e', 'goldstein', 'jeff'),
 ('davis', 'justin', 'a', 'drew', 'maj'),
 ('glover', 'vinson', 'n', 'lengyel', 'jason'),
 ('ambler', 'andrew', 's', 'office', 'visitors'),
 ('ambler', 'john', 's', 'office', 'visitors'),
 ('anderson', 'cindy', 'l', 'office', 'visitors'),
 ('anderson', 'wayne', 's', 'office', 'visitors'),
 ('andrade', 'andrea', 'm', 'office', 'visitors'),
 ('arcelle', 'jeanne', 'l', 'office', 'visitors'),
 ('arcelle', 'mark', '', 'office', 'visitors'),
 ('arnold', 'curtis', '', 'office', 'visitors'),
 ('baade', 'kraig', 'd',

In [21]:
filtered.count()

911249

In [22]:
filtered = filtered.filter(lambda x: x[0] != "")

In [23]:
filtered.count()

911249

# The number of records dropped

In [24]:
dropped = tokens2.count() - filtered.count()

In [25]:
dropped

59255

# The 10 most frequent visitors
(NAMELAST, NAMEFIRST, NAMEMID)
to the White House. 

(visitor) (frequency)

In [26]:
visitors = filtered.map(lambda x: ((x[0], x[1], x[2]), 1))

In [27]:
visitors = visitors.reduceByKey(lambda x,y: x+y)

In [28]:
#visitors.take(5)

In [29]:
top_10_visitors = visitors.takeOrdered(10, key = lambda x: -x[1])

In [30]:
top_10_visitors

[(('thomas', 'benjamin', 'l'), 185),
 (('berner', 'katherine', 'k'), 176),
 (('haas', 'jordan', 'm'), 152),
 (('grant', 'patrick', 'c'), 151),
 (('kidwell', 'lauren', 'k'), 145),
 (('haro', 'steven', 'm'), 140),
 (('garza', 'steven', 'a'), 127),
 (('strait', 'elan', ''), 107),
 (('lew', 'shoshana', 'm'), 102),
 (('zeitlin', 'daniel', 'l'), 98)]

# The 10 most frequently visited people
(visitee_namelast, visitee_namefirst)
in the White House.

(visitee) (frequency)

In [31]:
visitee = filtered.map(lambda x: ((x[3], x[4]), 1))

In [32]:
visitee.take(5)

[(('pelofsky', 'eric'), 1),
 (('baskerville', 'steven'), 1),
 (('baskerville', 'steven'), 1),
 (('baskerville', 'steven'), 1),
 (('baskerville', 'steven'), 1)]

In [33]:
visitee = visitee.reduceByKey(lambda x,y: x+y)

In [34]:
#visitee.take(5)

In [35]:
top_10_visitee = visitee.takeOrdered(10, key = lambda x: -x[1])

In [36]:
top_10_visitee

[(('office', 'visitors'), 430881),
 (('waves', 'visitorsoffice'), 44129),
 (('bryant', 'ruth'), 13970),
 (('oneil', 'olivia'), 13155),
 (('thompson', 'jared'), 11618),
 (('/', 'potus'), 10900),
 (('burton', 'collin'), 9672),
 (('megan', 'matthew'), 7944),
 (('mayerson', 'asher'), 6886),
 (('dessources', 'kalisha'), 5289)]

# The 10 most frequent visitor-visitee combinations.

(visitor-visitee) (frequency)

In [54]:
combo = filtered.map(lambda x: ((x[0], x[1], x[3], x[4]), 1))

In [55]:
combo.take(5)

[(('tajouribessassi', 'hanene', 'pelofsky', 'eric'), 1),
 (('bageant', 'laura', 'baskerville', 'steven'), 1),
 (('broemson', 'earl', 'baskerville', 'steven'), 1),
 (('jackling jr', 'william', 'baskerville', 'steven'), 1),
 (('mccrary', 'richard', 'baskerville', 'steven'), 1)]

In [39]:
combo = combo.reduceByKey(lambda x,y: x+y)

In [40]:
#combo.take(5)

In [50]:
top_10_combo = combo.takeOrdered(10, key = lambda x: -x[1])

In [51]:
top_10_combo

[(('kidwell', 'lauren', 'yudelson', 'alex'), 103),
 (('haas', 'jordan', 'yudelson', 'alex'), 90),
 (('grant', 'patrick', 'yudelson', 'alex'), 89),
 (('thomas', 'benjamin', 'yudelson', 'alex'), 89),
 (('cohen', 'mandy', 'lambrew', 'jeanne'), 84),
 (('haro', 'steven', 'yudelson', 'alex'), 84),
 (('berner', 'katherine', 'yudelson', 'alex'), 82),
 (('roche', 'shannon', 'yudelson', 'alex'), 70),
 (('urizar', 'jennifer', 'johnson', 'katie'), 68),
 (('martin', 'kathryn', 'lambrew', 'jeanne'), 61)]

In [43]:
tk = combo.takeOrdered(10)

In [45]:
tk

[(('aab', 'aaron', 'office', 'visitors'), 1),
 (('aab', 'sarah', 'office', 'visitors'), 1),
 (('aab', 'valerie', 'office', 'visitors'), 1),
 (('aabye', 'mary', 'office', 'visitors'), 1),
 (('aachal', 'suresh', 'office', 'visitors'), 1),
 (('aadnesen', 'christopher', 'office', 'visitors'), 1),
 (('aafreen', 'jahanzaib', 'office', 'visitors'), 1),
 (('aagaard', 'brian', 'pincus', 'erica'), 1),
 (('aagaard', 'lasse', 'zarek', 'cori'), 1),
 (('aagaard', 'nina', 'dworakfisher', 'keenan'), 1)]